In [1]:
import numpy as np
import pandas as pd

# not sure need matplot/ scipy/ openpyxl for this - just here because...
import matplotlib.pyplot as plt
import scipy as sp
import openpyxl

# these for scraping pdf's
import tabula as tb
import camelot


In [2]:
# define add to dictionary function
def add_allocations(anaesthetist, specialty, allocations):
    if pd.isna(anaesthetist):
        return allocations
    
    anaesthetist = anaesthetist.title()

    if anaesthetist in allocations.keys():
        allocations[anaesthetist].append(specialty)
    else:
        allocations[anaesthetist] = [specialty]
    return allocations


#extract and work with MOT part of map first
MAP_MOT = pd.read_excel('/home/christopher/Downloads/Theatre Daily map as excel sheet.xlsx', sheet_name = 0, header = 1, nrows = 9, engine = "openpyxl") #? drop columns 0,1/ 
allocations = dict()

#working with morning lists 
# get list of theaters and drop unnamed (empty) columns
theatres = [x for x in list(MAP_MOT) if 'Unnamed' not in x]
for theatre in theatres:
    specialty_am = MAP_MOT[theatre][0]
    anaesthetist_am = MAP_MOT[theatre][2]
    allocations = add_allocations(anaesthetist_am, specialty_am, allocations)

    specialty_pm = MAP_MOT[theatre][4]
    anaesthetist_pm = MAP_MOT[theatre][6]
    allocations = add_allocations(anaesthetist_pm, specialty_pm, allocations)

# extract and work with off-floor (lower) portion of map (otf -> 'off_the_floor')
MAP_OTF = pd.read_excel('/home/christopher/Downloads/Theatre Daily map as excel sheet.xlsx', sheet_name = 0, header = 1, skiprows = 10, nrows = 25, engine = "openpyxl") #? drop columns 0,1/ 
areas = [x for x in list(MAP_OTF)[0:8] if 'Unnamed' not in x]
for area in areas:
    specialty = area
    anaesthetist_am = MAP_OTF[area][0]
    allocations = add_allocations(anaesthetist_am, specialty, allocations)

    anaesthetist_pm = MAP_OTF[area][1]
    allocations = add_allocations(anaesthetist_pm, specialty, allocations)

allocations = add_allocations(MAP_OTF['Endo 1'][4], 'ECT', allocations)
allocations = add_allocations(MAP_OTF['Endo 2'][4], 'Echo', allocations)
allocations = add_allocations(MAP_OTF['Endo 3'][4], 'EP', allocations)
allocations = add_allocations(MAP_OTF['Endo 4'][4], 'Pain', allocations)
for col in list(MAP_OTF)[2:8]:
    allocations = add_allocations(MAP_OTF[col][8], 'Clinical Support', allocations)
    allocations = add_allocations(MAP_OTF[col][10], 'Clinical Support', allocations)
    allocations = add_allocations(MAP_OTF[col][15], 'Meetings', allocations)
    allocations = add_allocations(MAP_OTF[col][16], 'Meetings', allocations)


allocations

{'Kunju': ['Thoracics', 'Thoracics'],
 'Liew': ['Gen Surg', 'Gen Surg'],
 'Postle': ['Cardiac', 'Clinical Support'],
 'Cam Collard': ['Cardiac', 'Urology'],
 'Natani': ['Tavi'],
 'Beuth': ['Cardiac', 'Cardiac'],
 'Howarth': ['Cardiac', 'Cardiac'],
 'Marudachalam': ['NOF', 'Ortho'],
 'Stanton': ['Ortho', 'Ortho'],
 'Larsen': ['Ortho', 'Ortho'],
 'Mar': ['Gen Surg emerg', 'Gen Surg '],
 'Smith': ['Bronch', 'Clinical Support'],
 'Stonell': ['Endo 1', 'Endo 1'],
 'Toon': ['Endo 2', 'Endo 2'],
 'Closed': ['Endo 3', 'Endo 3', 'Echo'],
 'Bacajewski': ['Endo 4', 'Endo 4'],
 'Vaughan': ['DA', 'Clinic'],
 'Margale': ['DA', 'EP'],
 'Rapchuk': ['Clinic', 'Meetings'],
 'J. Jayamaha': ['ECT']}